In [16]:
%load_ext autoreload
%autoreload 2
import logging
import sys, os
import talib
sys.path.insert(0, os.path.abspath(""+"../Crawler"))
sys.path.insert(0, os.path.abspath(""+"../Simulator"))

from Packing import Data_preprocessing
import Simulator
from Component import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Simulator 讀取DATA
# sim = Simulator.Simulator(updatePKL = True, rebuildPKL = False) //default 
# updatePKL = true 會 update DATA
# rebuildPKL = True 會DATA全部重整理, 較慢, 但比較不會出錯(少整理DATA), 一般用Simulator.Simulator()即可
# sim = Simulator.Simulator(updatePKL = False) 只拿舊的資料(上次update後的)
sim = Simulator.Simulator()

# Read pickle 
load origin pkl file.
load finance data pkl file.


In [34]:
# sim.DATA可抓所有資料, sim.DATA的struct是dictionary, 
# 裡面可能還會有dict或是dataframe, 可用sim.DATA.keys()或sim.Print_data_columns() 看有哪些資料
# 要建立 buy 和 sell 條件才能回測, struct需要轉換成Component(), 用法 comp = Component(你的data)
# 要讀Component的data, comp.data即可

# 1. 大盤, Note: sim.DATA可抓所有資料
TW_Futures = sim.DATA['加權指數']

# 2. Buy signals
# talib_KD 可以產生K D data, struct為Component: 可試試type(slow_k)
slow_k, slow_d = talib_KD(sim.DATA['加權指數'])
# Component 也可做 & | < > = ...,etc的運算
buy = (slow_d < 20) & (slow_d < slow_k)

# 3. Sell signals
# assume sell signals為buy的後三天
sell = buy.shift(3)

In [47]:
# show the INFO
sim.Run?

In [50]:
# 4. Show the RESULTS and plot the Profit
sim.Run(buy = buy, sell = sell, buy_price = 'Close')  
# 4.1 Results: 整理表格
# 4.2 Plot : 1. 台指獲利為從起點買著台指放著
#            2. 績效評估
#            3. 虧損評估, 公式: 現有資金/曾經最大獲利值 (用期貨跑得數值會有點奇怪)

# -----------------------RESULTS------------------------ #
#                                                        #
#               Period =      2011-09-01 to 2019-01-15   #
#         Num of Trade =              54                 #
#               Profit =           14580                 #
#           Max Profit =           16060                 #
#           Min Profit =          -30220                 #
# Return On Investment =           48.6%                 #
#       Annualized ROI =           5.51%                 #
#        Ave_hold_days =             4.2                 #
#        Max_hold_days =               7                 #
#        Min_hold_days =               3                 #
#                                                        #
# ------------------------------------------------------ #

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [48]:
# 5. Plot
# 藍色為此策略的持有時間
# Setup: set buy, sell, tax signal
# Iplotly: plot
sim.Setup(buy = buy, sell = sell).Iplotly(start_date='2017-01-01',end_date='2018-12-06')

In [49]:
#Usage example 1: To plot OHLC_data & holding info
'''
1. 
sim.Setup(buy = buy, sell = sell)
sim.Iplotly(OHLC_data=TW_Futures, start_date='2017-01-01',end_date='2018-12-06')

2.
sim.Setup(buy = buy, sell = sell).Iplotly(OHLC_data=TW_Futures, start_date='2017-01-01',end_date='2018-12-06')

3.
sim.Run(buy = buy, sell = sell, buy_price = 'Close', return_iplot = False)
sim.Iplotly(OHLC_data=TW_Futures, start_date='2017-01-01',end_date='2018-12-06')
'''
#Usage example 2: To show the RESULTS & plot
'''
sim.Run(buy = buy, sell = sell, buy_price = 'Close')
'''

"\nsim.Run(buy = buy, sell = sell, buy_price = 'Close')\n"